In [1]:
import time

import numpy as np
import pandas as pd
import psutil
import torch
import torch.nn.functional as F
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv, GCNConv, SAGEConv
from torch_geometric.utils import to_undirected
from ucimlrepo import fetch_ucirepo

try:
    import pynvml
    pynvml.nvmlInit()
    NVML_AVAILABLE = True
except:
    NVML_AVAILABLE = False

/home/klema/miniconda3/envs/graphsage/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ucimlrepo import fetch_ucirepo

german_credit = fetch_ucirepo(id=144)

print("=== FEATURES ===")
print("Тип:", type(german_credit.data.features))
print("Форма:", german_credit.data.features.shape if hasattr(german_credit.data.features, 'shape') else 'N/A')
print("Колонки:", german_credit.data.features.columns.tolist() if hasattr(german_credit.data.features, 'columns') else 'N/A')
print("\nПример признаков:")
print(german_credit.data.features.head(2))

print("\n=== TARGETS ===")
print("Тип:", type(german_credit.data.targets))
print("Форма:", german_credit.data.targets.shape if hasattr(german_credit.data.targets, 'shape') else 'N/A')
print("Колонки:", german_credit.data.targets.columns.tolist() if hasattr(german_credit.data.targets, 'columns') else 'N/A')
print("Уникальные значения:", np.unique(german_credit.data.targets) if hasattr(german_credit.data.targets, 'values') else german_credit.data.targets)

ConnectionError: Error connecting to server

In [3]:
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.neighbors import kneighbors_graph
from torch_geometric.utils import to_undirected
import pandas as pd

# Загрузка
german_credit = fetch_ucirepo(id=144)
X = german_credit.data.features  # DataFrame 1000 x 20: Attribute1..Attribute20
y = german_credit.data.targets   # DataFrame 1000 x 1: колонка 'class'

# Целевая переменная: 1 → 0 (good), 2 → 1 (bad)
y = y['class'].values
y = np.where(y == 1, 0, 1)  # good=0, bad=1

# Согласно официальному описанию датасета (UCI):
# Категориальные атрибуты (по индексу, начиная с 1)
categorical_indices = [1, 3, 4, 6, 7, 9, 10, 12, 14, 15, 17, 19, 20]
# Числовые атрибуты
numerical_indices = [2, 5, 8, 11, 13, 16, 18]

# Формируем списки имён колонок
categorical_cols = [f'Attribute{i}' for i in categorical_indices]
numerical_cols = [f'Attribute{i}' for i in numerical_indices]

print(f"Категориальные признаки ({len(categorical_cols)}): {categorical_cols}")
print(f"Числовые признаки ({len(numerical_cols)}): {numerical_cols}")

# Обработка категориальных: преобразуем строки (типа 'A11') в индексы
X_cat_encoded = np.zeros((len(X), len(categorical_cols)), dtype=np.int64)
cat_dims = []

for i, col in enumerate(categorical_cols):
    le = LabelEncoder()
    # Все значения уже строки в формате 'A11', 'A12' и т.д.
    X_cat_encoded[:, i] = le.fit_transform(X[col])
    cat_dims.append(len(le.classes_))

# Обработка числовых: стандартизация
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X[numerical_cols])

# Построение графа (на основе всех признаков)
X_combined = np.hstack([X_num_scaled, X_cat_encoded])
k = 5
adj_matrix = kneighbors_graph(X_combined, k, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
edge_index = to_undirected(edge_index)

# Разделение на train/val/test с сохранением баланса классов
num_nodes = len(X)
indices = np.arange(num_nodes)
train_idx, temp_idx = train_test_split(indices, test_size=0.4, random_state=42, stratify=y)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42, stratify=y[temp_idx])

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

# Создание объекта данных с разделёнными признаками
data = Data(
    x_num=torch.tensor(X_num_scaled, dtype=torch.float),
    x_cat=torch.tensor(X_cat_encoded, dtype=torch.long),
    edge_index=edge_index,
    y=torch.tensor(y, dtype=torch.long),
    train_mask=train_mask,
    val_mask=val_mask,
    test_mask=test_mask
)

# Метаданные для моделей
data.num_numerical = X_num_scaled.shape[1]
data.num_categorical = len(categorical_cols)
data.cat_dims = cat_dims

print(f"\n✅ Загружено {num_nodes} узлов")
print(f"Числовые признаки: {data.num_numerical}, Категориальные: {data.num_categorical}")
print(f"Уникальные значения по кат. признакам: {cat_dims}")
print(f"Train/Val/Test: {train_mask.sum()}/{val_mask.sum()}/{test_mask.sum()}")
print(f"Распределение классов: {np.bincount(y)} (0=good, 1=bad)")

Категориальные признаки (13): ['Attribute1', 'Attribute3', 'Attribute4', 'Attribute6', 'Attribute7', 'Attribute9', 'Attribute10', 'Attribute12', 'Attribute14', 'Attribute15', 'Attribute17', 'Attribute19', 'Attribute20']
Числовые признаки (7): ['Attribute2', 'Attribute5', 'Attribute8', 'Attribute11', 'Attribute13', 'Attribute16', 'Attribute18']

✅ Загружено 1000 узлов
Числовые признаки: 7, Категориальные: 13
Уникальные значения по кат. признакам: [4, 5, 10, 5, 5, 4, 3, 4, 3, 3, 4, 2, 2]
Train/Val/Test: 600/200/200
Распределение классов: [700 300] (0=good, 1=bad)


In [5]:
# Вычисление весов перед обучением
y_train = data.y[data.train_mask].cpu().numpy()
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Используем устройство целевой переменной (всегда присутствует)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(data.y.device)

# Использование в лоссе
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

In [6]:
def compute_metrics(pred, true, probs=None):
    pred = pred.cpu().numpy()
    true = true.cpu().numpy()
    
    metrics = {
        'accuracy': (pred == true).mean(),
        'precision': precision_score(true, pred, zero_division=0),
        'recall': recall_score(true, pred, zero_division=0),
        'f1': f1_score(true, pred, zero_division=0),
    }
    
    if probs is not None:
        probs = probs.cpu().numpy()
        metrics['roc_auc'] = roc_auc_score(true, probs)
        metrics['pr_auc'] = average_precision_score(true, probs)
    
    return metrics

def accuracy(pred_y, y):
    return ((pred_y == y).sum() / len(y)).item()

def test(model, data):
    """Тестирование с поддержкой обеих сигнатур: (x, edge_index) и (x_num, x_cat, edge_index)"""
    model.eval()
    with torch.no_grad():
        # Автоматическое определение сигнатуры модели по количеству параметров в forward
        if hasattr(data, 'x_num') and hasattr(data, 'x_cat'):
            # Новая сигнатура с эмбеддингами
            _, out = model(data.x_num, data.x_cat, data.edge_index)
        else:
            # Старая сигнатура с объединёнными признаками
            _, out = model(data.x, data.edge_index)
        
        pred = out.argmax(dim=1)[data.test_mask]
        probs = torch.exp(out)[:, 1][data.test_mask]  # Вероятность класса 1 (дефолт)
        true = data.y[data.test_mask]
        
        return compute_metrics(pred, true, probs)

In [7]:
import torch.nn as nn

class FeatureEncoder(torch.nn.Module):
    def __init__(self, cat_dims, cat_emb_dims, num_numerical, output_dim):
        super().__init__()
        assert len(cat_dims) == len(cat_emb_dims)
        
        self.embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(num_categories, emb_dim)
            for num_categories, emb_dim in zip(cat_dims, cat_emb_dims)
        ])
        
        total_input_dim = sum(cat_emb_dims) + num_numerical
        self.projector = torch.nn.Sequential(
            torch.nn.Linear(total_input_dim, output_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
    
    def forward(self, x_num, x_cat):
        embeddings = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        x = torch.cat([x_num] + embeddings, dim=1)
        return self.projector(x)

In [13]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, cat_dims, cat_emb_dims, num_numerical, dim_h, dim_out):
        super().__init__()
        self.feature_encoder = FeatureEncoder(
            cat_dims=cat_dims,
            cat_emb_dims=cat_emb_dims,
            num_numerical=num_numerical,
            output_dim=dim_h
        )
        self.sage1 = SAGEConv(dim_h, dim_h)
        self.sage2 = SAGEConv(dim_h, dim_out)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)

    def forward(self, x_num, x_cat, edge_index):
        x = self.feature_encoder(x_num, x_cat)
        h = self.sage1(x, edge_index)
        h = torch.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.sage2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs, patience=20):
        # Взвешенный лосс для дисбаланса классов
        y_train = data.y[data.train_mask].cpu().numpy()
        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        weights = torch.tensor(weights, dtype=torch.float).to(data.x_num.device)
        criterion = torch.nn.CrossEntropyLoss(weight=weights)
        optimizer = self.optimizer

        # Ранняя остановка
        best_val_loss = float('inf')
        patience_counter = 0
        best_state = None

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            _, out = self(data.x_num, data.x_cat, data.edge_index)  # ← ключевое изменение
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                # Метрики на трейне
                train_pred = out[data.train_mask].argmax(dim=1)
                train_metrics = compute_metrics(train_pred, data.y[data.train_mask])
                
                # Метрики на валидации
                val_pred = out[data.val_mask].argmax(dim=1)
                val_probs = torch.exp(out)[:, 1][data.val_mask]
                val_metrics = compute_metrics(val_pred, data.y[data.val_mask], probs=val_probs)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Ранняя остановка
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_state = {k: v.cpu() for k, v in self.state_dict().items()}
            else:
                patience_counter += 1
                if patience_counter >= patience and epoch > 30:  # минимум 30 эпох обучения
                    print(f"\n⏹️  Early stopping at epoch {epoch} (best val loss: {best_val_loss:.3f})")
                    self.load_state_dict(best_state)
                    break

            if epoch % 10 == 0:
                print(f'Epoch {epoch:>3} | '
                    f'TL: {loss:.3f} | TF1: {train_metrics["f1"]:.3f} | '
                    f'VL: {val_loss:.3f} | VF1: {val_metrics["f1"]:.3f} | '
                    f'VRec: {val_metrics["recall"]:.3f} | VPR: {val_metrics["pr_auc"]:.3f}')
                
class GCN(torch.nn.Module):
    def __init__(self, cat_dims, cat_emb_dims, num_numerical, dim_h, dim_out):
        super().__init__()
        # Энкодер признаков: эмбеддинги + проекция
        self.feature_encoder = FeatureEncoder(
            cat_dims=cat_dims,
            cat_emb_dims=cat_emb_dims,
            num_numerical=num_numerical,
            output_dim=dim_h  # проецируем сразу в размерность первого GCN-слоя
        )
        
        # Графовые слои
        self.gcn1 = GCNConv(dim_h, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_out)
        
        # Оптимизатор с дифференцированной регуляризацией
        # (сильнее регуляризуем эмбеддинги для борьбы с переобучением)
        self.optimizer = torch.optim.Adam([
            {'params': self.feature_encoder.embeddings.parameters(), 'weight_decay': 1e-3},
            {'params': self.feature_encoder.projector.parameters(), 'weight_decay': 5e-4},
            {'params': self.gcn1.parameters(), 'weight_decay': 5e-4},
            {'params': self.gcn2.parameters(), 'weight_decay': 5e-4},
        ], lr=0.02)

    def forward(self, x_num, x_cat, edge_index):
        # Шаг 1: кодирование признаков через эмбеддинги
        x = self.feature_encoder(x_num, x_cat)
        
        # Шаг 2: графовая обработка (сохраняем архитектуру оригинальной GCN)
        h = F.dropout(x, p=0.5, training=self.training)
        h = self.gcn1(h, edge_index)
        h = torch.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.gcn2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs, patience=20):
        # Взвешенный лосс для дисбаланса классов
        y_train = data.y[data.train_mask].cpu().numpy()
        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        weights = torch.tensor(weights, dtype=torch.float).to(data.x_num.device)
        criterion = torch.nn.CrossEntropyLoss(weight=weights)
        optimizer = self.optimizer

        # Ранняя остановка
        best_val_loss = float('inf')
        patience_counter = 0
        best_state = None

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            _, out = self(data.x_num, data.x_cat, data.edge_index)  # ← ключевое изменение
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                # Метрики на трейне
                train_pred = out[data.train_mask].argmax(dim=1)
                train_metrics = compute_metrics(train_pred, data.y[data.train_mask])
                
                # Метрики на валидации
                val_pred = out[data.val_mask].argmax(dim=1)
                val_probs = torch.exp(out)[:, 1][data.val_mask]
                val_metrics = compute_metrics(val_pred, data.y[data.val_mask], probs=val_probs)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Ранняя остановка
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_state = {k: v.cpu() for k, v in self.state_dict().items()}
            else:
                patience_counter += 1
                if patience_counter >= patience and epoch > 30:  # минимум 30 эпох обучения
                    print(f"\n⏹️  Early stopping at epoch {epoch} (best val loss: {best_val_loss:.3f})")
                    self.load_state_dict(best_state)
                    break

            if epoch % 10 == 0:
                print(f'Epoch {epoch:>3} | '
                    f'TL: {loss:.3f} | TF1: {train_metrics["f1"]:.3f} | '
                    f'VL: {val_loss:.3f} | VF1: {val_metrics["f1"]:.3f} | '
                    f'VRec: {val_metrics["recall"]:.3f} | VPR: {val_metrics["pr_auc"]:.3f}')

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

def monitor_resources():
    stats = {}
    # CPU & RAM
    stats['ram_mb'] = psutil.virtual_memory().used / (1024 ** 2)
    stats['cpu_percent'] = psutil.cpu_percent()

    # GPU
    if device.type == 'cuda' and NVML_AVAILABLE:
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        util = pynvml.nvmlDeviceGetUtilizationRates(handle)
        stats['gpu_mem_mb'] = mem_info.used / (1024 ** 2)
        stats['gpu_util'] = util.gpu
    else:
        stats['gpu_mem_mb'] = None
        stats['gpu_util'] = None
    return stats

def train_with_monitoring(model, data, epochs, model_name):
    print(f"\n{'='*50}\nTraining {model_name} with resource monitoring\n{'='*50}")
    
    if device.type == 'cuda':
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()

    start_time = time.time()

    model.fit(data, epochs)

    final_ram = psutil.virtual_memory().used / (1024 ** 2)
    max_gpu_mem = None
    if device.type == 'cuda':
        max_gpu_mem = torch.cuda.max_memory_allocated() / (1024 ** 2)

    duration = time.time() - start_time
    test_metrics = test(model, data)

    results = {
        'test_metrics': test_metrics,
        'training_time_sec': duration,
        'final_ram_mb': final_ram,
        'max_gpu_mem_mb': max_gpu_mem,
    }

    print(f"\n{model_name} finished")
    print(f"Test Recall: {test_metrics.get('recall', 0):.3f} | "
          f"F1: {test_metrics.get('f1', 0):.3f} | "
          f"PR-AUC: {test_metrics.get('pr_auc', 0):.3f}")
    print(f"Training Time: {duration:.1f} sec")
    if max_gpu_mem:
        print(f"Peak GPU Memory: {max_gpu_mem:.1f} MB")
    print(f"Final RAM Usage: {final_ram:.1f} MB")

    return test_metrics.get('recall', 0), results

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

# Адаптивные размерности эмбеддингов: √(уникальных значений) + небольшой запас
cat_emb_dims = [
    min(16, max(4, int(np.sqrt(dim)) + 2))
    for dim in data.cat_dims
]

print(f"Размерности эмбеддингов: {cat_emb_dims}")
print(f"Суммарная размерность после конкатенации: {sum(cat_emb_dims) + data.num_numerical}")

# Инициализация модели
graphsage = GraphSAGE(
    cat_dims=data.cat_dims,
    cat_emb_dims=cat_emb_dims,
    num_numerical=data.num_numerical,
    dim_h=64,
    dim_out=2
).to(device)

# Обучение (функция train_with_monitoring без изменений)
recall_sage, results_sage = train_with_monitoring(
    graphsage, data, epochs=100, model_name="GraphSAGE (с эмбеддингами)"
)

Размерности эмбеддингов: [4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
Суммарная размерность после конкатенации: 60

Training GraphSAGE (с эмбеддингами) with resource monitoring
Epoch   0 | TL: 0.698 | TF1: 0.327 | VL: 0.709 | VF1: 0.315 | VRec: 0.333 | VPR: 0.332
Epoch  10 | TL: 0.584 | TF1: 0.615 | VL: 0.578 | VF1: 0.595 | VRec: 0.733 | VPR: 0.574
Epoch  20 | TL: 0.526 | TF1: 0.640 | VL: 0.553 | VF1: 0.605 | VRec: 0.767 | VPR: 0.660
Epoch  30 | TL: 0.417 | TF1: 0.714 | VL: 0.570 | VF1: 0.595 | VRec: 0.733 | VPR: 0.556

⏹️  Early stopping at epoch 31 (best val loss: 0.532)

GraphSAGE (с эмбеддингами) finished
Test Recall: 0.733 | F1: 0.607 | PR-AUC: 0.617
Training Time: 0.4 sec
Final RAM Usage: 5729.7 MB


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

# Адаптивные размерности эмбеддингов: √(уникальных значений) + небольшой запас
cat_emb_dims = [
    min(16, max(4, int(np.sqrt(dim)) + 2))
    for dim in data.cat_dims
]

print(f"Размерности эмбеддингов: {cat_emb_dims}")
print(f"Суммарная размерность после конкатенации: {sum(cat_emb_dims) + data.num_numerical}")

# Инициализация модели
graphsage = GCN(
    cat_dims=data.cat_dims,
    cat_emb_dims=cat_emb_dims,
    num_numerical=data.num_numerical,
    dim_h=64,
    dim_out=2
).to(device)

# Обучение (функция train_with_monitoring без изменений)
recall_sage, results_sage = train_with_monitoring(
    graphsage, data, epochs=100, model_name="GCN (с эмбеддингами)"
)

Размерности эмбеддингов: [4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
Суммарная размерность после конкатенации: 60

Training GCN (с эмбеддингами) with resource monitoring
Epoch   0 | TL: 0.718 | TF1: 0.463 | VL: 0.718 | VF1: 0.436 | VRec: 0.850 | VPR: 0.300
Epoch  10 | TL: 0.651 | TF1: 0.529 | VL: 0.647 | VF1: 0.533 | VRec: 0.867 | VPR: 0.528
Epoch  20 | TL: 0.590 | TF1: 0.574 | VL: 0.558 | VF1: 0.577 | VRec: 0.683 | VPR: 0.642
Epoch  30 | TL: 0.553 | TF1: 0.611 | VL: 0.534 | VF1: 0.607 | VRec: 0.733 | VPR: 0.684
Epoch  40 | TL: 0.562 | TF1: 0.605 | VL: 0.577 | VF1: 0.582 | VRec: 0.683 | VPR: 0.625
Epoch  50 | TL: 0.536 | TF1: 0.616 | VL: 0.548 | VF1: 0.609 | VRec: 0.817 | VPR: 0.652

⏹️  Early stopping at epoch 53 (best val loss: 0.522)

GCN (с эмбеддингами) finished
Test Recall: 0.650 | F1: 0.582 | PR-AUC: 0.578
Training Time: 0.9 sec
Final RAM Usage: 5731.0 MB
